In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
# Your custom text
custom_text = """
hello how are you
i am learning machine learning
machine learning is fun
deep learning is advanced
artificial intelligence is the future
"""
# Function to train LSTM on any text
def train_lstm_predictor(text_data, epochs=50):
    # Tokenize
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([text_data])
    total_words = len(tokenizer.word_index) + 1 
    # Create sequences
    sentences = text_data.strip().split('\n')
    sequences = []
    for sentence in sentences:
        words = sentence.split()
        for i in range(1, len(words)):
            seq = words[:i+1]
            sequences.append(' '.join(seq))
    # Convert to numerical
    seq_numeric = tokenizer.texts_to_sequences(sequences)
    # Pad sequences
    max_len = max([len(seq) for seq in seq_numeric])
    padded = pad_sequences(seq_numeric, maxlen=max_len, padding='pre')
    # Split X and y
    X = padded[:, :-1]
    y = padded[:, -1]
    # Build model
    model = Sequential([
        Embedding(total_words, 10, input_length=max_len-1),
        LSTM(20),
        Dense(total_words, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, epochs=epochs, verbose=0)
    return model, tokenizer, max_len
# Train model
model, tokenizer, max_len = train_lstm_predictor(custom_text)
# Predict function
def predict_next(sentence):
    # Convert to sequence
    seq = tokenizer.texts_to_sequences([sentence])[0]
    # Pad
    padded_seq = pad_sequences([seq], maxlen=max_len-1, padding='pre')
    # Predict
    pred = model.predict(padded_seq, verbose=0)
    predicted_index = np.argmax(pred)
    # Find word
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            return word
    return ""
# Test with user input
user_input = input("Enter a sentence: ")
predicted_word = predict_next(user_input)
print(f"\nInput: {user_input}")
print(f"Predicted next word: {predicted_word}")
print(f"Complete: {user_input} {predicted_word}")